In [41]:
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_decision_regions
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

In [42]:
employee_df = pd.read_csv(r'C:\Users\jakob\Desktop\Projektseminar\Data\employees.csv', delimiter= '|')
absences_df = pd.read_csv(r'C:\Users\jakob\Desktop\Projektseminar\Data\absences.csv', delimiter='|')


In [ ]:
#absences_df = absences_df.head(1000)

In [43]:
absences_df = absences_df.dropna(subset=['start_date', 'end_date'])
absences_df = absences_df[absences_df['start_date'].str.contains('2024') == False]


In [ ]:
# Step 1: Create an empty list to store the rows of the new DataFrame
new_rows = []

# Step 2: Iterate over the original DataFrame row by row
for index, row in absences_df.iterrows():
    # Step 3: Generate a list of dates between 'start_date' and 'end_date' (inclusive)
    date_range = pd.date_range(start=row['start_date'], end=row['end_date'])
    
    # Step 4: Duplicate the row for each date in the list and add a new column with the date value
    for date in date_range:
        new_row = row[['employee_id', 'reason', 'allocation_id']].copy()
        new_row['date'] = date
        new_rows.append(new_row)

# Step 5: Append the duplicated rows to the empty list

# Step 6: Create a new DataFrame using the list of duplicated rows
new_df = pd.DataFrame(new_rows)

# Reset the index of the new DataFrame
new_df.reset_index(drop=True, inplace=True)

#ssave the shit
new_df.to_csv(r'C:\Users\jakob\Desktop\Projektseminar\Data\absences_ezrows.csv', index=False)

# Display the new DataFrame
new_df


In [51]:
new_df = pd.read_csv(r'C:\Users\jakob\Desktop\Projektseminar\Data\absences_ezrows.csv', delimiter = ',')
new_df

,employee_id,reason,allocation_id,date
0,9827,vacation,98044.0,2020-01-17
1,9827,vacation,98044.0,2020-01-18
2,9827,vacation,118649.0,2020-01-17
3,9827,vacation,118649.0,2020-01-18
4,1366,vacation,2679.0,2020-01-20
...,...,...,...,...
301975,9521,vacation,NaN,2022-11-29
301976,9521,vacation,NaN,2022-11-30
301977,9521,vacation,NaN,2022-12-01
301978,9521,vacation,NaN,2022-12-02


In [52]:
new_df = pd.read_csv(r'C:\Users\jakob\Desktop\Projektseminar\Data\absences_ezrows.csv', delimiter = ',')
new_df

# order new_df by date
new_df = new_df.sort_values(by=['date'])

min_date = new_df['date'].min()
max_date = new_df['date'].max()
# Create a DataFrame with the date_range
date_range = pd.date_range(start=min_date, end=max_date)
all_dates = pd.DataFrame({'Date': date_range})
all_dates

,Date
0,2020-01-02
1,2020-01-03
2,2020-01-04
3,2020-01-05
4,2020-01-06
...,...
1462,2024-01-03
1463,2024-01-04
1464,2024-01-05
1465,2024-01-06


In [ ]:
# make 'date' in new_df to datetime
# show all unique values in new_df date


In [53]:
# join the two DataFrames
merged_df = pd.merge(all_dates, new_df, how='left', left_on='Date', right_on='date')

#drop Date and employee_id
#merged_df = merged_df.drop(['Date', 'employee_id'], axis=1)
merged_df

ValueError: You are trying to merge on datetime64[ns] and object columns. If you wish to proceed you should use pd.concat

In [ ]:
# count the number of absences per day in a new column 'count'
merged_df['count'] = merged_df.groupby('date')['Date'].transform('count')
#drop duplicates of column 'Date'
merged_df = merged_df.drop_duplicates(subset=['date'])
merged_df['day_of_week'] = merged_df['date'].dt.dayofweek
merged_df['month'] = merged_df['date'].dt.month
merged_df['season'] = merged_df['date'].dt.quarter
merged_df

In [ ]:
# how many NaN values are in the column 'coun'
merged_df['count'].isna().sum()
# fill NaN with mean of 'count'
merged_df['count'] = merged_df['count'].fillna(merged_df['count'].mean())
merged_df

In [ ]:
#save new_df as csv
merged_df.to_csv(r'C:\Users\jakob\Desktop\Projektseminar\Data\absences_new_for_darts.csv', index=False)

In [ ]:
# Split the dataset into features (X) and target variable (y)
X = new_df[['day_of_week', 'month']]
y = new_df['count']

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the random forest regression model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model on the training set
rf_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)

In [ ]:
# Split the dataset into features (X) and target variable (y)
X = new_df[['day_of_week', 'month']]
y = new_df['count']

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the random forest regression model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model on the training set
rf_model.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R-squared:", r2)